In [14]:
%matplotlib notebook

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy import stats

In [16]:
#Load FEMA Data Sets
owner_data = pd.read_csv('Data Sets/Owners Export_cleaned.csv', encoding='cp1252')
renters_data = pd.read_csv('Data Sets/Renters Export_cleaned.csv')

In [17]:
#Visualize Dataframes
owner_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4447,OH,Auglaize (County),CRIDERSVILLE,45806,1,$-,1,$-,1,...,0,0,$0,$0,$0,$0,0,0,0,0
1,4447,OH,Auglaize (County),NEW KNOXVILLE,45871,1,"$2,376.23",1,"$2,376.23",0,...,0,1,$697.52,$698,$0,$0,1,0,0,0
2,4447,OH,Auglaize (County),SAINT MARYS,45885,3,$-,3,$-,3,...,0,0,$0,$0,$0,$0,0,0,0,0
3,4447,OH,Auglaize (County),WAPAKONETA,45895,21,"$6,735.89",14,"$94,302.44",4,...,1,5,"$56,475.44","$41,315","$2,406","$12,754",3,1,1,0
4,4447,OH,Darke (County),ARCANUM,45304,1,$-,1,$-,1,...,0,0,$0,$0,$0,$0,0,0,0,0


In [18]:
#Visualize Dataframes
renters_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4454,TX,Cameron (County),BROWNSVILLE,78521,2,0,0,0,0,0,0,$-,$-,$-,$0,0,0,0,0
1,4454,TX,Cameron (County),BROWNSVILLE,78526,1,1,1,0,0,0,0,$-,$-,$-,$0,0,0,0,0
2,4454,TX,Cameron (County),CAMERON,76520,1,1,1,0,0,0,0,$-,$-,$-,$0,0,0,0,0
3,4454,TX,Cameron (County),COMBES,78535,7,7,6,1,0,0,2,"$4,197.14",$-,"$1,436.00","$2,761",2,0,0,0
4,4454,TX,Cameron (County),HARLINGEN,78550,224,179,110,69,0,0,94,"$227,526.33",$-,"$102,614.12","$124,912",94,0,0,0


In [19]:
#Make copy and clean data sets
converted_renters_data = renters_data.copy()
converted_renters_data['Total Approved IHP Amount'] = converted_renters_data['Total Approved IHP Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_renters_data['Rental Amount'] = converted_renters_data['Rental Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_renters_data['Repair/Replace Amount'] = converted_renters_data['Repair/Replace Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_renters_data['Other Needs Amount'] = converted_renters_data['Other Needs Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')




In [20]:
converted_renters_data.head(10)

,Disaster,State,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4454,TX,Cameron (County),BROWNSVILLE,78521,2,0,0,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
1,4454,TX,Cameron (County),BROWNSVILLE,78526,1,1,1,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
2,4454,TX,Cameron (County),CAMERON,76520,1,1,1,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
3,4454,TX,Cameron (County),COMBES,78535,7,7,6,1,0,0,2,4197.14,0.0,1436.00,2761.0,2,0,0,0
4,4454,TX,Cameron (County),HARLINGEN,78550,224,179,110,69,0,0,94,227526.33,0.0,102614.12,124912.0,94,0,0,0
5,4454,TX,Cameron (County),HARLINGEN,78552,204,173,95,78,0,0,96,290237.67,0.0,134241.00,155997.0,95,1,0,0
6,4454,TX,Cameron (County),LA FERIA,78559,63,47,29,18,0,0,25,55019.78,0.0,25700.00,29320.0,25,0,0,0
7,4454,TX,Cameron (County),PALM VALLEY,78552,5,5,4,1,0,0,1,1126.00,0.0,1126.00,0.0,1,0,0,0
8,4454,TX,Cameron (County),PORT ISABEL,78578,1,0,0,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
9,4454,TX,Cameron (County),RIO HONDO,78583,1,1,1,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0


In [21]:
#Makes copy and clean data set
converted_owner_data = owner_data.copy()
converted_owner_data["Total Approved IHP Amount"] = converted_owner_data["Total Approved IHP Amount"].str.replace(',','').str.replace('$','').astype('float')
converted_owner_data['Average FEMA Inspected Damage'] = converted_owner_data["Average FEMA Inspected Damage"].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Rental Amount'] = converted_owner_data['Rental Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Repair/Replace Amount'] = converted_owner_data['Repair/Replace Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Other Needs Amount'] = converted_owner_data['Other Needs Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Approved between $1 and $10,000'] = converted_owner_data['Approved between $1 and $10,000'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Approved between $10,001 and $25,000'] = converted_owner_data['Approved between $10,001 and $25,000'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Approved between $25,001 and Max'] = converted_owner_data['Approved between $25,001 and Max'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Total Max Grants'] = converted_owner_data['Total Max Grants'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Total Damage'] = converted_owner_data['Total Damage'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')


In [22]:
converted_owner_data.head(10)

,Disaster,State,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4447,OH,Auglaize (County),CRIDERSVILLE,45806,1,0.00,1,0.00,1,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4447,OH,Auglaize (County),NEW KNOXVILLE,45871,1,2376.23,1,2376.23,0,...,0,1,697.52,698.0,0.0,0.0,1.0,0.0,0.0,0.0
2,4447,OH,Auglaize (County),SAINT MARYS,45885,3,0.00,3,0.00,3,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4447,OH,Auglaize (County),WAPAKONETA,45895,21,6735.89,14,94302.44,4,...,1,5,56475.44,41315.0,2406.0,12754.0,3.0,1.0,1.0,0.0
4,4447,OH,Darke (County),ARCANUM,45304,1,0.00,1,0.00,1,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4447,OH,Darke (County),GREENVILLE,45331,3,0.00,2,0.00,2,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4447,OH,Darke (County),NEW MADISON,45346,12,500.18,4,2000.70,1,...,0,1,588.80,589.0,0.0,0.0,1.0,0.0,0.0,0.0
7,4447,OH,Greene (County),BEAVER CREEK,45431,2,0.00,1,0.00,1,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4447,OH,Greene (County),BEAVERCREEK,45431,265,1594.41,41,65370.64,9,...,0,4,10202.18,6648.0,3304.0,250.0,4.0,0.0,0.0,0.0
9,4447,OH,Greene (County),BEAVERCREEK,45432,5,70.94,2,141.87,1,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
#Count the number of inputs by state
state_counts_rent = converted_renters_data["State"].value_counts()
state_counts

NameError: name 'state_counts' is not defined

In [24]:
#Count the number of inputs by state
state_counts_owners = converted_owner_data["State"].value_counts()
state_counts_owners

PR    7649
FL    4878
TX    3919
NC    1483
LA    1147
SC    1123
OK     933
MO     920
MP     882
IL     840
CA     654
GA     464
AR     435
MI     345
VI     308
WV     303
MS     293
NE     250
CO     243
AL     230
IN     174
KY     161
SD     145
OH     116
IA     110
MN      77
VA      73
AK      69
HI      57
WA      27
AS      19
TN      11
WY      10
Name: State, dtype: int64

In [25]:
# Using GroupBy in order to separate the data into fields according to "state" values
grouped_renters_df = converted_renters_data.groupby(['State'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_renters_df)

# In order to be visualized, a data function must be used...
grouped_renters_df.count().head(10)

,Disaster,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
State,,,,,,,,,,,,,,,,,,,
AK,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40
AL,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162
AR,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259
AS,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
CA,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619
CO,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198
FL,5774,5774,5771,5774,5774,5774,5774,5774,5774,5774,5774,5774,5774,5774,5774,5774,5774,5774,5774
GA,422,422,422,422,422,422,422,422,422,422,422,422,422,422,422,422,422,422,422
HI,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49


In [26]:
# Using GroupBy in order to separate the data into fields according to "state" values
grouped_owners_df = converted_owner_data.groupby(['State'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_owners_df)

# In order to be visualized, a data function must be used...
grouped_owners_df.count().head(10)

,Disaster,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,"FEMA Inspected Damage between $1 and $10,000",...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
State,,,,,,,,,,,,,,,,,,,,,
AK,69,69,69,69,69,69,69,69,69,69,...,69,69,69,69,69,69,69,69,69,69
AL,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,230,230,230
AR,435,435,435,435,435,435,435,435,435,435,...,435,435,435,435,435,435,435,435,435,435
AS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
CA,654,654,654,654,654,654,654,654,654,654,...,654,654,654,654,654,654,654,654,654,654
CO,243,243,243,243,243,243,243,243,243,243,...,243,243,243,243,243,243,243,243,243,243
FL,4878,4878,4878,4878,4878,4878,4878,4878,4878,4878,...,4878,4878,4878,4878,4878,4878,4878,4878,4878,4878
GA,464,464,464,464,464,464,464,464,464,464,...,464,464,464,464,464,464,464,464,464,464
HI,57,57,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57


In [27]:
count = grouped_owners_df['Disaster'].count()
total = grouped_owners_df['Total Approved IHP Amount'].sum()
grouped_owners_df['Total Approved IHP Amount'].sum()
state_summary_table_owners = pd.DataFrame({"Number of Claims": count,
                                    "Total Approved IHP Amount": total})

In [28]:
state_summary_table_owners

,Number of Claims,Total Approved IHP Amount
State,,
AK,69,2.958174e+07
AL,230,1.709113e+07
AR,435,1.646970e+07
AS,19,1.860799e+07
CA,654,8.124703e+07
CO,243,5.325217e+07
FL,4878,6.748345e+08
GA,464,2.422787e+07
HI,57,1.129281e+07


In [29]:
count1 = grouped_renters_df['Disaster'].count()
total1 = grouped_renters_df['Total Approved IHP Amount'].sum()
grouped_renters_df['Total Approved IHP Amount'].sum()
state_summary_table_renters = pd.DataFrame({"Number of Claims": count1,
                                    "Total Approved IHP Amount": total1})

In [30]:
state_summary_table_combined = state_summary_table_renters.copy()

In [31]:
state_summary_table_combined ['Number of Claims Owners'] = state_summary_table_owners['Number of Claims']
state_summary_table_combined ['Total Approved IHP Amount Owners'] = state_summary_table_owners['Total Approved IHP Amount']
state_summary_table_combined ['Number of Claims'] = state_summary_table_owners['Number of Claims'] + state_summary_table_combined ['Number of Claims Owners']
state_summary_table_combined ['Total Approved IHP Amount'] = state_summary_table_owners['Total Approved IHP Amount'] + state_summary_table_combined ['Total Approved IHP Amount Owners']
state_summary_table_combined = state_summary_table_combined.drop(['Number of Claims Owners','Total Approved IHP Amount Owners'], axis=1)

In [32]:
state_summary_table_combined.head()

,Number of Claims,Total Approved IHP Amount
State,,
AK,138,5.916348e+07
AL,460,3.418227e+07
AR,870,3.293939e+07
AS,38,3.721598e+07
CA,1308,1.624941e+08


In [65]:
converted_owner_data_slimed = converted_owner_data[['State','County','Total Approved IHP Amount']]
converted_renters_data_slimed = converted_owner_data[['State','County','Total Approved IHP Amount']]
converted_combined_data_slimed = pd.merge(converted_owner_data_slimed, converted_renters_data_slimed, how='outer' )
converted_combined_data_slimed = converted_combined_data_slimed.drop_duplicates(subset=None, keep='first', inplace=False)
county_summary_table = converted_combined_data_slimed.groupby(['State','County']).sum()

In [66]:
county_summary_table

Total Approved IHP Amount
State County                                                                       
AK    Alaska Gateway Regional Educational Attendance ...                   68623.95
      Anchorage (Borough)                                               19801204.61
      Kenai Peninsula (Borough)                                           587825.63
      Lower Yukon Regional Educational Attendance Area                    445503.04
      Matanuska-Susitna (Borough)                                        6217654.42
...                                                                             ...
WV    Summers (County)                                                    376390.06
      Webster (County)                                                   2405660.11
      Wetzel (County)                                                     887050.49
WY    Johnson (County)                                                    101064.34
      Niobrara (County)                                                  1072226.06

[870 rows x 1 columns]